In [1]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time

  # Replace with your file's ID
#list of articles
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
#list of districts and alloted funds
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
# past public data
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"


inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"
master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [2]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [3]:
def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [4]:
articles_df = read_file(article_data_id)
# Find invalid article names
def valid_articles(one):
    invalid_articles = set(one) - set(articles_df["Articles"])

    # Output result
    if invalid_articles:
        print("These articles are not in the master list:")
        print("\n".join(invalid_articles))
    else:
        print("All articles are valid.")


Download progress: 100%


## District


In [5]:
district_df = read_file(master_data_id)
district_df


Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Ariyalur,Agri Battery Sprayer,Article,7,4200.0,29400.0,176000.0,NaN,No,District
1,Ariyalur,Canon Printer 6030B (USB),Article,1,13900.0,13900.0,NaN,NaN,No,District
2,Ariyalur,Desktop Computer,Article,1,45500.0,45500.0,NaN,NaN,No,District
3,Ariyalur,Sewing Machine ORD,Article,15,5200.0,78000.0,NaN,9200.0,No,District
4,Chengalpattu,Agri Power Sprayer (1.2 Bolo),Article,2,11760.0,23520.0,160000.0,NaN,No,District
...,...,...,...,...,...,...,...,...,...,...
110,Thiruppur,Education Aid,Aid,1,50000.0,50000.0,440000.0,NaN,L. Swetha. (B.E),District
111,Thiruppur,Education Aid,Aid,1,15000.0,15000.0,NaN,NaN,M. Abinaya (BSc),District
112,Thiruppur,Education Aid,Aid,1,15000.0,15000.0,NaN,360000.0,M. Madhumitha (BSc),District
113,Tirupathur,Ex gratia for Deceased,Aid,2,25000.0,50000.0,585000.0,NaN,"For 2 families, each Rs.25000",District


In [6]:
district_df["REQUESTED ARTICLE"].nunique()

42

In [7]:
district_req = district_df.groupby("REQUESTED ARTICLE",as_index=False)["QUANTITY"].sum()

district_req

,REQUESTED ARTICLE,QUANTITY
0,Agri Battery Sprayer,59
1,Agri Power Sprayer (1.2 Bolo),15
2,Aluminium Idli Making Box,3
3,Aluminium Vessels Sets,2
4,Audio Mixer,1
5,Barber Set,1
6,Bosch Electrician Kit 10 Re,3
7,Bosch Electrician Kit 13 Re,2
8,Bosch Rotary Hammer GBH 220,1
9,Business Aid,1


### Public

In [8]:
public_df = read_file(public_master_id)
public_df = public_df.rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,REQUESTED ARTICLE,Cost Per Unit,QUANTITY,Total Value,Comments,Beneficiary Type
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,15000,1,15000,Rs.15000 for children,Public
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public
...,...,...,...,...,...,...,...,...,...,...,...,...
100,P 101,M.Kumar,612081203379,No,"Ozkalur, Kancheepuram",9597058028,Business Aid,10000,1,10000,No,Public
101,P 102,M.Vinayagam,498101932330,Yes,"Ozhalur, Chenglepat",7418225869,Financial Aid,0,1,0,25% of the cost of Handicapped scooter,Public
102,P 103,D.Yuvasree,726646590112,No,"Arakonam, Ranipet",9500061246,Education Aid,100000,1,100000,No,Public
103,P 104,T.selvi,411012263584,No,"Melakalangal, Tenkasi",9751217032,Construction Aid,20000,1,20000,No,Public


In [9]:
public_df["REQUESTED ARTICLE"].nunique()

23

In [10]:
valid_articles(public_df["REQUESTED ARTICLE"])

All articles are valid.


In [11]:
public_req = public_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
public_req

,REQUESTED ARTICLE,QUANTITY
0,Bosch Electrician Kit 13 Re,1
1,Business Aid,16
2,Construction Aid,6
3,Education Aid,7
4,Electronic weighing scale,1
5,Financial Aid,5
6,Gents Cycle,4
7,Girls Cycle,4
8,Goat,2
9,Hand Sewing Machine with Motor,2


### Institution

In [12]:
inst = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})
inst_df = inst[inst["Beneficiary Type"]=="Institution"]
inst_df

Download progress: 100%


,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Unnamed: 9,Beneficiary Type
0,I 001,Cheyyur Thasildar Office,Desktop Computer,1,45500,45500,"Thasildar Office, Cheyyur, Chengalpattu",0,needs high config PC for e-seva point,NaN,Institution


In [13]:
valid_articles(inst_df["REQUESTED ARTICLE"])

All articles are valid.


In [14]:
inst_df["REQUESTED ARTICLE"].nunique()

1

In [15]:
inst_req = inst_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
inst_req

,REQUESTED ARTICLE,QUANTITY
0,Desktop Computer,1


### Others

In [16]:
oth_df = inst[inst["Beneficiary Type"]=="Others"]
oth_df

,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Unnamed: 9,Beneficiary Type


In [17]:
valid_articles(oth_df["REQUESTED ARTICLE"])

All articles are valid.


In [18]:
oth_df["REQUESTED ARTICLE"].nunique()

0

In [19]:
oth_req = oth_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
oth_req

,REQUESTED ARTICLE,QUANTITY


In [20]:
total_qty = pd.concat([district_req, public_req,inst_req,oth_req]).groupby('REQUESTED ARTICLE', as_index=False).sum()
total_qty

,REQUESTED ARTICLE,QUANTITY
0,Agri Battery Sprayer,59
1,Agri Power Sprayer (1.2 Bolo),15
2,Aluminium Idli Making Box,3
3,Aluminium Vessels Sets,2
4,Audio Mixer,1
5,Barber Set,1
6,Bosch Electrician Kit 10 Re,3
7,Bosch Electrician Kit 13 Re,3
8,Bosch Rotary Hammer GBH 220,1
9,Business Aid,17
